# TOPIC MODELLING

In [342]:
#importing libraries
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import statistics
import math
import os
import re
import string
import numpy as np
from gensim import models,corpora
import nltk
from nltk.corpus import stopwords
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import re
pd.set_option('max_colwidth', 800)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [343]:
#loading dataset
df=pd.read_excel('/content/PMO translated Cleaned data.xlsx')
df.head()

,Survey Answer ID,Q10\nIs there anything else you would like to share with us?,Q11 translated,Q11\nno-translated
0,f91da0c6-602e-40b0-a96e-aef7469e6ebb,Compliments,thanks,thanks
1,3a836d06-fb1c-41a6-99aa-39202beb281c,Comment on the Service,good,good
2,7f6a288d-d2db-42df-9ee2-aba2b0a782ca,Comment on the Service,NICE SERVICES,NICE SERVICESS
3,c3e1f8d4-2956-4734-9571-6c31656484be,Comment on the Service,page response is unstable. rest is all fine,page respsonse is unstable. rest is all fine
4,a3d6ce90-d5e4-4214-8f14-3899d34e8980,Comment on the Service,THANKS,THANKS


In [344]:
df.shape

(29220, 4)

In [345]:
df.columns

Index(['Survey Answer ID',
       'Q10\nIs there anything else you would like to share with us?',
       'Q11 translated', 'Q11\nno-translated'],
      dtype='object')

In [346]:
### dropping unnecessary columns (id and no translated)
df = df.drop(['Survey Answer ID', 'Q10\nIs there anything else you would like to share with us?', 'Q11\nno-translated'], axis=1)
df.head()

,Q11 translated
0,thanks
1,good
2,NICE SERVICES
3,page response is unstable. rest is all fine
4,THANKS


In [347]:
# renaming the column
df.rename(columns = {'Q11 translated': 'text'}, inplace = True)
df.head()

,text
0,thanks
1,good
2,NICE SERVICES
3,page response is unstable. rest is all fine
4,THANKS


### DATA PREPROCESSING AND EDA

In [348]:
# Cleaning Raw tweets
def clean_text(text):
    
    #remove emails
    text = ' '.join([i for i in text.split() if '@' not in i])
    
    #remove web address
    text = re.sub('http[s]?://\S+', '', text)

    #remove brackets and parantheses (and removing the text inside the brackets and parantheses)
    text = re.sub("[\(\[].*?[\)\]]", "", text)

    #remove punctuations
    text = re.sub(r'[^\w\s]', ' ', text)
    
    #Filter to allow only alphabets and numbers
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    #remove double spaces 
    text = re.sub('\s+', ' ', text)

    #remove more than 15 characters
    text = re.sub(r'\b\w{16,}\b', '', text)
    
    return text

In [349]:
# applying that into function.
df["clean_text"] = df['text'].apply(lambda x: clean_text(x))
df.head(10)

,text,clean_text
0,thanks,thanks
1,good,good
2,NICE SERVICES,NICE SERVICES
3,page response is unstable. rest is all fine,page response is unstable rest is all fine
4,THANKS,THANKS
5,thanks,thanks
6,System error message,System error message
7,Good,Good
8,page response is unstable. rest is all fine,page response is unstable rest is all fine
9,VERY GOOD AND WE ARE HAPPY KEEP THE GOOD THING GOING GOOD DAY,VERY GOOD AND WE ARE HAPPY KEEP THE GOOD THING GOING GOOD DAY


In [350]:
# dropping the uncleaned text column "text"
df = df.drop(['text'], axis=1)

In [351]:
# renaming the column
df.rename(columns = {'clean_text':'text'}, inplace = True)
df.head()

,text
0,thanks
1,good
2,NICE SERVICES
3,page response is unstable rest is all fine
4,THANKS


In [352]:
import nltk
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
print(sw_nltk)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [353]:
# preprocessing the data (removing stopwords, convert the sentence to lower case, and also applying lemmatization to get meaningful words )
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wnl=WordNetLemmatizer()
def clean_text(review):
  review = review.lower()
  review = review.split()
  review = [wnl.lemmatize(word) for word in review if word not in sw_nltk]
  #review = [wnl.lemmatize(word) for word in review if word not in sw_spacy]
  review = [wnl.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
  review = ' '.join(review)
  return review

In [354]:
df['text']=df['text'].apply(clean_text)
df.head()

,text
0,thanks
1,good
2,nice service
3,page response unstable rest fine
4,thanks


In [355]:
# function to remove names
def remove_names(text):
    # define regular expression pattern to match names
    pattern = re.compile(r'\b(?:abd|abdouli|abdul|abdulaziz|adel|abdullah|abdulrahman|abrar|abeer|abi|adn|adu|ahmad|ahmed|ahood|aht|alabdooli|alabdouli|alachram|alaghbari|alc|alhamdulillah|alhamoor|ali|alia|alif|alkindi|allah|allhumdulliah|allo|alqubaisi|alsalaam|alsoor|alykom|amal|amata|amed|amen|amer|amis|ammar|amoudi|amra|amsat|amwal|ana|andrew|anf|anz|aqsa|atta|atleat|asharj|ashkram|asif|asim|asma|asmaa|awael|awatef|ayash|ayisha|ayman|azhar|azza|badr|badria|badriya|bahia|bakr|balakrishnan|balooshi|balushi|baniyas|bara|baraem|barsh|bastaki|batayneh|bilal|bint|biyyat|brian|bthe|budaiwi|issy|ive|jaddaf|jafar|jafiliya|jahili|jalal|jamal||janan|janoubi|jasmine|jassim|jazak|jazeera|jebel|jeel|jihadjoseline|juma|jumaia|jvc|kalba|kalban|kareem|katyn|kawkaba|keji|kenneth|khabisi|khaili|khaimah|khair|khalaf|khaled|khalid|khalifa|khamis|kharma|khatib|khatm|khattab|khazna|khor|khorfakkan|kjhgfds|kkk|kmz|konain|kumz|kutty|kvito|kyrgyz|dadsad|sddaddad|sdsds||sha|shaer|shafei|shah|shahama|shakhbout|shama|shamikha|shamkha|shamsa|shamsi|shanahan|sharifa|sharq|shatha|shawamekh|shd|shefaa|shehhi|sheikh|sheikha|shiva|shkran|soo|sooo|soooo|sooooo|soooooo|soumya|sadsadsad|sssasss|suhail|suhaila|suhair|sukkah|szhp|taha|tahaluf|taharba|umm|umrah|xia|yabladi|yas|yasmine|yassin|yousef|youssef|youssefni|ypu|yuen|zaabi|zahir|zakher|zaki|zarooni|zakat|zayed|zina|zindabad|ziyad|zubaida|zuhair)\b', re.IGNORECASE)
    return re.sub(pattern, '', text)

# apply the function to the text column
df['text'] = df['text'].apply(remove_names)

In [356]:
# function to remove places
def replace_places(text):
    # define regular expression pattern to match places
    pattern = re.compile(r'\b(?:abu dhabi|abu|dhabi)\b', re.IGNORECASE)
    
    return re.sub(pattern, 'abudhabi', text)

# apply the function to the text column
df['text'] = df['text'].apply(replace_places)

In [357]:
# function to remove places
def replace_places(text):
    # define regular expression pattern to match places
    
    pattern = re.compile(r'\b(?:thank|thankyou)\b', re.IGNORECASE)
    
    return re.sub(pattern, 'thanks', text)

# apply the function to the text column
df['text'] = df['text'].apply(replace_places)

In [358]:
# removing single characters after cleaning the sentence
df['text'] = df['text'].str.replace(r'\b\w{1,2}\b', '', regex=True)
df.head()

,text
0,thanks
1,good
2,nice service
3,page response unstable rest fine
4,thanks


In [359]:
# Remove empty rows
df = df[df['text'].apply(lambda x: len(x) > 0)]
df.head()

,text
0,thanks
1,good
2,nice service
3,page response unstable rest fine
4,thanks


In [360]:
df.reset_index(inplace=True)

In [361]:
df['text'][16]

'thanks'

In [362]:
df.shape

(26609, 2)

In [363]:
# A function to prepare the text for topic modelling
def words(text):
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\r\t\n]')
    text = regex.sub(" ", text.lower())
    words = text.split(" ")
    words = [re.sub('\S*@\S*\s?', '', sent) for sent in words]
    words = [re.sub('\s+', ' ', sent) for sent in words]
    words = [re.sub("\'", "", sent) for sent in words]
    words = [w for w in words if not len(w) < 2]
    return words

In [364]:
nltk.download('wordnet')
docs = [words(x) for x in df['text']]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [374]:
docs[0]

['thanks']

In [367]:
df.shape

(26609, 2)

In [368]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

Number of unique words in initital documents: 4939


In [369]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print(len(corpus))

26609


In [370]:
bow_doc_20 = corpus[19]

for i in range(len(bow_doc_20)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_20[i][0], 
                                                     dictionary[bow_doc_20[i][0]], 
                                                     bow_doc_20[i][1]))

Word 28 ("satisfied") appears 1 time.


In [371]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 5);
        word_dict['Topic ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return word_dict;

In [372]:
import gensim
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word = dictionary,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=500,
                                           passes=5,
                                           alpha='auto',
                                           per_word_topics=True)

### Top 5 topics

In [373]:
get_lda_topics(lda_model, 5)     

{'Topic 01': ['service', 'excellent', 'great', 'fast', 'approval'],
 'Topic 02': ['good', 'happy', 'best', 'god', 'bless'],
 'Topic 03': ['may', 'reward', 'time', 'please', 'transaction'],
 'Topic 04': ['thanks', 'much', 'nothing', 'effort', 'easy'],
 'Topic 05': ['nice', 'satisfied', 'none', 'payment', 'super']}